In [1]:
import torch
import numpy as np
import copy

import random
import math
import time
import pickle
import ray

from finitefield.finitefield import FiniteField
from welchberlekamp import makeEncoderDecoder


In [87]:
def RS_reconstruct_secret(encoded_shares, Field, solveSystem):
    encoded_msg = []
    for share in encoded_shares:
        x, y = share[0], share[1]
        encoded_msg.append( [Field(x),Field(y)] )
    Q,E = solveSystem(encoded_msg)
    P, remainder = (Q.__divmod__(E))
    out = int(P(0))
    return out

def isprime(n):
    if n == 2:
        return True
    if n == 1 or n % 2 == 0:
        return False
    i = 3
    while i <= math.sqrt(n):
        if n % i == 0:
            return False
        i = i + 2
    return True


def initial(Z_lower=100):
    # generate q bigger than z_lower
    q = Z_lower
    while True:
        if isprime(q):
            break
        else:
            q = q + 1
    print("q = " + str(q))
    print("\nq is prime\n")

    # Find p and r
    r = 1
    while True:
        p = r * q + 1
        if isprime(p):
            print("r = " + str(r))
            print("p = " + str(p))
            print("\np is prime\n")
            break
        r = r + 1

    # Compute elements of Z_p*
    Z_p_star = []
    for i in range(0, p):
        if (math.gcd(i, p) == 1):
            Z_p_star.append(i)
        if len(Z_p_star) > 100:
            break

    # print("Z_p* = ")
    # print(Z_p_star) # , len(Z_p_star) same length, i.e. range(p)

    # Compute elements of G = {h^r mod p | h in Z_p*}
    G = []
    for i in Z_p_star:
        G.append((i ** r) % p)

    G = list(set(G))
    G.sort()
    # print("\nG = ")
    # print(G)
    print("Order of G is " + str(len(G)) + ". This must be equal to q:",q)

    # Since the order of G is prime, any element of G except 1 is a generator
    g = random.choice(list(filter(lambda g: g != 1, G)))
    print("\ng = " + str(g) + "\n")

    return p, q, r, g

def generate_shares(n, t, secret, p, q, r, g, idxs, flag):
#     if secret==0: secret=1 # one exception after quantization
    assert secret >= 0 and secret < q, "secret not in range "+str(secret)

    FIELD_SIZE = q
    coefficients = coeff(t, secret, FIELD_SIZE)
    # print("coefficients",coefficients,type(coefficients[0]))
    users = list(idxs) # users are to recieve the shares
    assert n==len(users), "these two number should be identical"

    shares = []
    for i in users:
        i = int(i)
        f_i = f(i, coefficients, q)
        shares.append((i, f_i))

    if flag==1:
        # start = time.time()
        commitments = commitment(coefficients, g, p)
        # print("commitments:", commitments)
        # print("commitments take ", time.time()-start,"seconds")

        verifications = []
        for i in users:
            i = int(i)
            check1 = quick_pow(g, share_ith(shares, i), p)
            # check1 = g ** share_ith(shares, i) % p
            check2 = verification(g, commitments, i, p, q)
            verifications.append(check2)
            if (check1 % p)  == (check2  % p) :
                pass
            else:
                # print(g, share_ith(shares, i), p , q, i, commitments, shares)
                print("checking fails with:", check1, check2)
                1/0
            # print(i, "-th user ============= tag at time ", time.time()-start,"seconds =============")
            # start = time.time()
    else:
        commitments, verifications = [0,], [0,]
    return shares, commitments, verifications

def generate_shares_LCC(N, T, K, secrets, p, q, r, g, alphas, betas, flag):
#     secrets_check = []
#     for secret in secrets:
#         if secret == 0:
#             secrets_check.append(1)  # "0": one exception after quantization
#         else:
#             secrets_check.append(secret)
#     secrets = secrets_check
    for secret in secrets:
        assert secret >= 0 and secret < q, "secret not in range"

    FIELD_SIZE = q
    noises = [random.randrange(0, FIELD_SIZE) for _ in range(T)]

    shares = []
    for alpha in alphas:
        y = _lagrange_interpolate(alpha, betas, secrets + noises, q)
        shares.append((alpha, y))
    
    if flag==1:
        commitments = commitment(secrets + noises, g, p)
        start = time.time()

        verifications = []
        for alpha in alphas:
            # check1 = g ** shares[i-1][1] % p
            # check1 = g ** share_ith(shares, i) % p
            check1 = quick_pow(g, share_ith(shares, alpha), p)
            check2 = verification(commitments, alpha, betas, p, q)
            verifications.append(check2)
            if (check1 % p) == (check2 % p):
                pass
            else:
                print("checking fails with:", check1, check2)
                1/0
            print(alpha, "-th user ============= tag at time ", time.time()-start,"seconds =============")
            start = time.time()
    else:
        commitments, verifications = [0,], [0,]
    return shares, commitments, verifications

def coeff(t, secret, FIELD_SIZE):
    coeff = [random.randrange(0, FIELD_SIZE) for _ in range(t - 1)]
    coeff.append(secret)  # a0 is secret
    return coeff

def f(x, coefficients, q):
    # y = Decimal('0')
    # for coefficient_index, coefficient_value in enumerate(coefficients[::-1]):
    #     y += (Decimal(str(x)) ** Decimal(str(coefficient_index)) * Decimal(str(coefficient_value)))
    #     y = Decimal(int(y)%q)
    # return int(y)
    y = 0
    for coefficient_index, coefficient_value in enumerate(coefficients[::-1]):
        y += x ** coefficient_index * coefficient_value
        y = int(y) % q
    return y

def commitment(coefficients, g, p):
    commitments = []
    for coefficient_index, coefficient_value in enumerate(coefficients[::-1]):
        # c = g ** coefficient_value % p
        c = quick_pow(g, coefficient_value, p)
        commitments.append(c)
    return commitments

def verification(g, commitments, i, p, q):
    v = 1
    for k, c in enumerate(commitments):
        # v = ( v * (c ** ((i ** k) % q)) ) % p
        v = (v * quick_pow(c, (i ** k) % q , p)) % p
    return v

def quick_pow(a, b, q):  # compute a^b mod q, in a faster way？
    temp = 1
    for i in range(1, b + 1):
        temp = (temp * a) % q
    return temp % q

def recursive_pow(a, b, q): # recursive powering in finite field
    # let a, b be integers
    if a == 0:
        return 0
    if b == 0:
        return 1
    elif b > 0:
        if b%2 == 0: # b is even
            return recursive_pow(a*a % q, b/2, q) % q
        elif b%2 == 1: # b is odd
            return a * recursive_pow(a*a % q, (b-1)/2, q) % q
    return 0

def reconstruct_secret(pool, q):
    x_s,y_s = [],[]
    for share in pool:
        x_s.append(int(share[0]))
        y_s.append(int(share[1]))
    out = _lagrange_interpolate(0, x_s, y_s, q)
    return out

def reconstruct_secret_LCC(pool, q, betas, K):
    out = []
    x_s, y_s = [], []
    for share in pool:
        x_s.append(int(share[0]))
        y_s.append(int(share[1]))
    for k in range(K):
        beta = betas[k]
        # out.append(f_rec(beta,pool,q))
        out.append(_lagrange_interpolate(beta, x_s, y_s, q))
    return out

def _lagrange_interpolate(x, x_s, y_s, p):
    """
    Find the y-value for the given x, given n (x, y) points;
    k points will define a polynomial of up to kth order.
    """
    k = len(x_s)
    assert k == len(set(x_s)), "points must be distinct"

    def PI(vals):  # upper-case PI -- product of inputs
        accum = 1
        for v in vals:
            accum *= v
        return accum

    nums = []  # avoid inexact division
    dens = []
    for i in range(k):
        others = list(x_s)
        cur = others.pop(i)
        nums.append(PI(int(x - o) for o in others))
        dens.append(PI(int(cur - o) for o in others))
    den = PI(dens)
    num = sum([_divmod(int(int(nums[i]) * int(den) * int(y_s[i]) % p), int(dens[i]), p) for i in range(k)])
    # debug: overflow,  ^ here  :  cast to int
    # num = 0
    # for i in range(k):
    #     temp = int(int(nums[i]) * int(den) * int(y_s[i]) % p)
    #     num += _divmod(temp, int(dens[i]), p)

    return (_divmod(num, den, p) + p) % p

def _extended_gcd(a, b):
    """
    Division in integers modulus p means finding the inverse of the
    denominator modulo p and then multiplying the numerator by this
    inverse (Note: inverse of A is B such that A*B % p == 1) this can
    be computed via extended Euclidean algorithm
    http://en.wikipedia.org/wiki/Modular_multiplicative_inverse#Computation
    """
    x = 0
    last_x = 1
    y = 1
    last_y = 0
    while b != 0:
        quot = a // b
        a, b = b, a % b
        x, last_x = last_x - quot * x, x
        y, last_y = last_y - quot * y, y
    return int(last_x), int(last_y)

def _divmod(num, den, p):
    """Compute num / den modulo prime p

    To explain what this means, the return value will be such that
    the following is true: den * _divmod(num, den, p) % p == num
    """

    inv, _ = _extended_gcd(den, p)
    return num * inv

def share_ith(shares, i):
    for share in shares:
        if share[0] == i:
            return share[1]
    return None

In [88]:
n = 2
t = 2
p,q,r,g = initial(13)
print('p,q,r,g:',p,q,r,g)
users = [1,2] # list(range(1,11))
secret = 2

shares = generate_shares(n, t, secret, p, q, r, g, users, flag=0) # flag 1 : skip verification step
print(shares) # ( secret shares, commit, veri)
shares = shares[0]

Field = FiniteField(p=q)
# _, _, solveSystem1 = makeEncoderDecoder(n, t, q)
# RS_reconstruct_secret(shares, Field, solveSystem1)

recon_secret = reconstruct_secret(shares,q)
print(recon_secret)

q = 13

q is prime

r = 4
p = 53

p is prime

Order of G is 13. This must be equal to q: 13

g = 46

p,q,r,g: 53 13 4 46
([(1, 8), (2, 1)], [0], [0])
2


In [89]:
# beaver's protocol
# consider SSS
# [y]i * [z]i = [yz]i

p,q,r,g = initial(13)
Field = FiniteField(p=q)
print('p,q,r,g:',p,q,r,g)
y,z = 2,3
a,b,c = 3,5,int(Field(3*5))

shares,_,_ = generate_shares(2, 2, y, p, q, r, g, [1,2], flag=0)
y1,y2 = Field(shares[0][1]), Field(shares[1][1])
shares,_,_ = generate_shares(2, 2, z, p, q, r, g, [1,2], flag=0)
z1,z2 = Field(shares[0][1]), Field(shares[1][1])
shares,_,_ = generate_shares(2, 2, a, p, q, r, g, [1,2], flag=0)
a1,a2 = Field(shares[0][1]), Field(shares[1][1])
shares,_,_ = generate_shares(2, 2, b, p, q, r, g, [1,2], flag=0)
b1,b2 = Field(shares[0][1]), Field(shares[1][1])
shares,_,_ = generate_shares(2, 2, c, p, q, r, g, [1,2], flag=0)
c1,c2 = Field(shares[0][1]), Field(shares[1][1])

d1, e1 = y1-a1, z1-b1
d2, e2 = y2-a2, z2-b2
d, e = reconstruct_secret([(1,d1),(2,d2)], q), reconstruct_secret([(1,e1),(2,e2)], q)
sig1 = d*e + d*b1+e*a1+c1 # share 1: [yz]1
sig2 = d*e + d*b2+e*a2+c2 # share 2: [yz]2
print("sig1,sig2:",sig1,sig2)
shares = [(1,sig1),(2,sig2)]
recon = reconstruct_secret(shares, q)
print("recon:",recon)

q = 13

q is prime

r = 4
p = 53

p is prime

Order of G is 13. This must be equal to q: 13

g = 42

p,q,r,g: 53 13 4 42
sig1,sig2: 0 7
recon: 6


In [90]:
def beaver_sss(y1,z1,y2,z2,q):
    Field = FiniteField(p=q)
    a,b,c = 3,5,int(Field(3*5))
    shares,_,_ = generate_shares(2, 2, a, p, q, r, g, [1,2], flag=0)
    a1,a2 = Field(shares[0][1]), Field(shares[1][1])
    shares,_,_ = generate_shares(2, 2, b, p, q, r, g, [1,2], flag=0)
    b1,b2 = Field(shares[0][1]), Field(shares[1][1])
    shares,_,_ = generate_shares(2, 2, c, p, q, r, g, [1,2], flag=0)
    c1,c2 = Field(shares[0][1]), Field(shares[1][1])
    d1, e1 = y1-a1, z1-b1
    d2, e2 = y2-a2, z2-b2
    d, e = reconstruct_secret([(1,d1),(2,d2)], q), reconstruct_secret([(1,e1),(2,e2)], q)
    yz1 = d*e + d*b1+e*a1+c1 # share 1: [yz]1
    yz2 = d*e + d*b2+e*a2+c2 # share 2: [yz]2
    return yz1,yz2

y,z = 2,3
shares,_,_ = generate_shares(2, 2, y, p, q, r, g, [1,2], flag=0)
y1,y2 = Field(shares[0][1]), Field(shares[1][1])
shares,_,_ = generate_shares(2, 2, z, p, q, r, g, [1,2], flag=0)
z1,z2 = Field(shares[0][1]), Field(shares[1][1])
yz1,yz2 = beaver_sss(y1,z1,y2,z2,q)
yz1,yz2,reconstruct_secret([(1,yz1),(2,yz2)], q)

(3 (mod 13), 0 (mod 13), 6)

In [93]:
# beaver's protocol
# consider SSS: when n is bigger than 2
# this case should be suitable for LCC
# [y]i * [z]i = [yz]i

p,q,r,g = initial(13)
Field = FiniteField(p=q)
print('p,q,r,g:',p,q,r,g)
y,z = 2,3
a,b,c = 3,5,int(Field(3*5))
n,t = 3,2

shares,_,_ = generate_shares(n, t, y, p, q, r, g, [1,2,3], flag=0)
y1,y2,y3 = Field(shares[0][1]), Field(shares[1][1]), Field(shares[2][1])
shares,_,_ = generate_shares(n, t, z, p, q, r, g, [1,2,3], flag=0)
z1,z2,z3 = Field(shares[0][1]), Field(shares[1][1]), Field(shares[2][1])
shares,_,_ = generate_shares(n, t, a, p, q, r, g, [1,2,3], flag=0)
a1,a2,a3 = Field(shares[0][1]), Field(shares[1][1]), Field(shares[2][1])
shares,_,_ = generate_shares(n, t, b, p, q, r, g, [1,2,3], flag=0)
b1,b2,b3 = Field(shares[0][1]), Field(shares[1][1]), Field(shares[2][1])
shares,_,_ = generate_shares(n, t, c, p, q, r, g, [1,2,3], flag=0)
c1,c2,c3 = Field(shares[0][1]), Field(shares[1][1]), Field(shares[2][1])

d1, e1 = y1-a1, z1-b1
d2, e2 = y2-a2, z2-b2
d3, e3 = y3-a3, z3-b3
d, e = reconstruct_secret([(1,d1),(2,d2),(3,d3)], q), reconstruct_secret([(1,e1),(2,e2),(3,e3)], q)
sig1 = d*e + d*b1+e*a1+c1 # share 1: [yz]1
sig2 = d*e + d*b2+e*a2+c2 # share 2: [yz]2
sig3 = d*e + d*b3+e*a3+c3 # share 3: [yz]3
print("sig1,sig2:",sig1,sig2)
shares = [(1,sig1),(2,sig2),(3,sig3)]
recon = reconstruct_secret(shares, q)
print("recon:",recon)

q = 13

q is prime

r = 4
p = 53

p is prime

Order of G is 13. This must be equal to q: 13

g = 28

p,q,r,g: 53 13 4 28
sig1,sig2: 11 3
recon: 6


In [99]:
# SNIP with SSS: case 1
from finitefield.polynomial import polynomialsOver
Poly = polynomialsOver(Field)

x1, x2, x3 = 1,2,3
u1,u2 = 2,2 # （1）
v1,v2 = 3,6 # （2）

# compute poly over u1,u2 ; v1,v2
poly_f = Poly([2,0]) # depending on (1)
poly_g = Poly([0,3]) # depending on (2)
assert poly_f(1) == u1 and poly_f(2) == u2
assert poly_g(1) == v1 and poly_g(2) == v2
print(poly_f(1),poly_f(2),poly_g(1),poly_g(2))
h = poly_f*poly_g
print(h)
h_coefs = h.coefficients
h_shares = []
for c in h_coefs:
    shares,_,_ = generate_shares(2, 2, int(c), p, q, r, g, [1,2], flag=0)
    h_shares.append(shares)
x_shares = []
for x in [x1,x2,x3]:
    shares,_,_ = generate_shares(2, 2, x, p, q, r, g, users, flag=0)
    x_shares.append(shares)
    
print("h_shares",h_shares,'x_shares',x_shares)
poly_h1 = Poly([h_shares[0][0][1], h_shares[1][0][1]])
poly_h2 = Poly([h_shares[0][1][1], h_shares[1][1][1]])
print("h shares:",poly_h1, ';',poly_h2)

# client idx = 1
print("**client 1**")
u11 = x_shares[1][0][1]
u21 = x_shares[0][0][1] +1
v11 = x_shares[2][0][1]
v21 = poly_h1(1)
print("f1:",u11,u21," g1:",v11,v21)

# client idx = 2
print("**client 2**")
u12 = x_shares[1][1][1]
u22 = x_shares[0][1][1] +1
v12 = x_shares[2][1][1]
v22 = poly_h2(1)
print("f2:",u12,u22," g2:",v12,v22)

# polynomial check
r = 2 # any number

# shares = [(1,poly_h1(2)), (2, poly_h2(2))]
# recon = reconstruct_secret(shares, q)
# print("recon at out:",recon)

# Use Beaver
y1 = _lagrange_interpolate(r, [1,2], [u11,u21], q) 
y2 = _lagrange_interpolate(r, [1,2], [u12,u22], q) 
z1 = _lagrange_interpolate(r, [1,2], [v11,v21], q)
z2 = _lagrange_interpolate(r, [1,2], [v12,v22], q)

yz1,yz2 = beaver_sss(y1,z1,y2,z2,q)
yz_recon = reconstruct_secret([(1,yz1),(2,yz2)], q)
check = yz_recon - reconstruct_secret([(1,poly_h1(r)), (2,poly_h2(r))], q)
print("check",check)

2 2 3 6
0 + 6 t^1
h_shares [[(1, 5), (2, 10)], [(1, 10), (2, 1)]] x_shares [[(1, 8), (2, 2)], [(1, 6), (2, 10)], [(1, 8), (2, 0)]]
h shares: 5 + 10 t^1 ; 10 + 1 t^1
**client 1**
f1: 6 9  g1: 8 2
**client 2**
f2: 10 3  g2: 0 11
check 0


In [100]:
# SNIP with SSS: case 2
# another input x
from finitefield.polynomial import polynomialsOver
Poly = polynomialsOver(Field)

x1, x2, x3 = 3,2,1
u1,u2 = 2,6 # （1）
v1,v2 = 1,2 # （2）

# compute poly over u1,u2 ; v1,v2
poly_f = Poly([11,4]) # depending on (1)
poly_g = Poly([0,1]) # depending on (2)
assert poly_f(1) == u1 and poly_f(2) == u2
assert poly_g(1) == v1 and poly_g(2) == v2
print(poly_f(1),poly_f(2),poly_g(1),poly_g(2))
h = poly_f*poly_g
print(h) # notice: degree of 2
h_coefs = h.coefficients
h_shares = []
for c in h_coefs:
    shares,_,_ = generate_shares(2, 2, int(c), p, q, r, g, [1,2], flag=0)
    h_shares.append(shares)
x_shares = []
for x in [x1,x2,x3]:
    shares,_,_ = generate_shares(2, 2, x, p, q, r, g, users, flag=0)
    x_shares.append(shares)
    
print("h_shares",h_shares,'x_shares',x_shares)
poly_h1 = Poly([h_shares[0][0][1], h_shares[1][0][1], h_shares[2][0][1]])
poly_h2 = Poly([h_shares[0][1][1], h_shares[1][1][1], h_shares[2][1][1]])
print("h shares:",poly_h1, ';',poly_h2)

# client idx = 1
print("**client 1**")
u11 = x_shares[1][0][1]
u21 = x_shares[0][0][1] *2
v11 = x_shares[2][0][1]
v21 = poly_h1(1)
print("f1:",u11,u21," g1:",v11,v21)

# client idx = 2
print("**client 2**")
u12 = x_shares[1][1][1]
u22 = x_shares[0][1][1] *2
v12 = x_shares[2][1][1]
v22 = poly_h2(1)
print("f2:",u12,u22," g2:",v12,v22)

# polynomial check
r = 1

# Use Beaver
y1 = _lagrange_interpolate(r, [1,2], [u11,u21], q) 
y2 = _lagrange_interpolate(r, [1,2], [u12,u22], q) 
z1 = _lagrange_interpolate(r, [1,2], [v11,v21], q)
z2 = _lagrange_interpolate(r, [1,2], [v12,v22], q)

yz1,yz2 = beaver_sss(y1,z1,y2,z2,q)
yz_recon = reconstruct_secret([(1,yz1),(2,yz2)], q)
check = yz_recon - reconstruct_secret([(1,poly_h1(r)), (2,poly_h2(r))], q)
print("check",check)

2 6 1 2
0 + 11 t^1 + 4 t^2
h_shares [[(1, 6), (2, 12)], [(1, 5), (2, 12)], [(1, 12), (2, 7)]] x_shares [[(1, 0), (2, 10)], [(1, 4), (2, 6)], [(1, 0), (2, 12)]]
h shares: 6 + 5 t^1 + 12 t^2 ; 12 + 12 t^1 + 7 t^2
**client 1**
f1: 4 0  g1: 0 10
**client 2**
f2: 6 20  g2: 12 5
check 0


In [136]:
# SNIP with LCC:
from finitefield.polynomial import polynomialsOver
Poly = polynomialsOver(Field)

T, N, K = 2, 4, 2
alphas = list(range(1, 1+N))
betas = list(range(1+N, N+K+T+1))

# polynomial check
r = 1

# first circuit
x11, x12, x13 = 1,2,3
u11,u12 = 2,2 # （1）
v11,v12 = 3,6 # （2）

# compute poly over u1,u2 ; v1,v2
poly_f1 = Poly([2,0]) # depending on (1)
poly_g1 = Poly([0,3]) # depending on (2)
assert poly_f1(1) == u11 and poly_f1(2) == u12
assert poly_g1(1) == v11 and poly_g1(2) == v12
print(poly_f1(1),poly_f1(2),poly_g1(1),poly_g1(2))
h1 = poly_f1*poly_g1
print(h1)
h1_coefs = h1.coefficients
h1_coefs.append(0) # degree 2 term

# second circuit
x21, x22, x23 = 3,2,1
u21,u22 = 2,6 # （1）
v21,v22 = 1,2 # （2）

# compute poly over u1,u2 ; v1,v2
poly_f2 = Poly([11,4]) # depending on (1)
poly_g2 = Poly([0,1]) # depending on (2)
assert poly_f2(1) == u21 and poly_f2(2) == u22
assert poly_g2(1) == v21 and poly_g2(2) == v22
print(poly_f2(1),poly_f2(2),poly_g2(1),poly_g2(2))
h2 = poly_f2*poly_g2
print(h2) # notice: degree of 2
h2_coefs = h2.coefficients

users_wb = {1: {}, 2: {}, 3: {}, 4: {}}
for i in range(len(h2_coefs)):
    secrets = [int(h1_coefs[i]), int(h2_coefs[i])]
    shares,_,_ = generate_shares_LCC(N, T, K, secrets, p, q, r, g, alphas, betas, flag=0)
    for key in users_wb.keys():
        users_wb[key]['h_shares_{}'.format(i)] = share_ith(shares, key)
X = [[x11,x21],[x12,x22],[x13,x23]]
for i in range(3):
    secrets = [int(X[i][0]), int(X[i][1])]
    shares,_,_ = generate_shares_LCC(N, T, K, secrets, p, q, r, g, alphas, betas, flag=0)
    for key in users_wb.keys():
        users_wb[key]['x_shares_{}'.format(i)] = share_ith(shares, key)
    
print("users' shares:",users_wb[1])
for key in users_wb.keys():
    print("**client {}**".format(key))
    user = users_wb[key]
    
    poly_h = Poly([user['h_shares_0'],user['h_shares_1'],user['h_shares_2']])
    
    u1 = user['x_shares_{}'.format(1)]
    u2 = user['x_shares_{}'.format(0)] *2
    v1 = user['x_shares_{}'.format(2)]
    v2 = poly_h(1)
    print("f1:",u1,u2," g1:",v1,v2)
    
    users_wb[key]['u1'],users_wb[key]['u2'],users_wb[key]['v1'],users_wb[key]['v2'] = u1,u2,v1,v2
    
    poly_h_at_r = poly_h(r)
    users_wb[key]['h_r'] = poly_h_at_r

# polynomial check
print("users' shares:",users_wb[3])
for key in users_wb.keys():
    # beaver
    user = users_wb[key]
    y = _lagrange_interpolate(r, [1,2], [user['u1'],user['u2']], q) 
    z = _lagrange_interpolate(r, [1,2], [user['v1'],user['v2']], q) 
    users_wb[key]['y'], users_wb[key]['z'] = y,z

# beaver: server side
print("users' shares:",users_wb[2])
a,b,c = 3,5,int(Field(3*5))
n,t = N,T
a_shares,_,_ = generate_shares_LCC(N, T, K, [a]*K, p, q, r, g, alphas, betas, flag=0)
b_shares,_,_ = generate_shares_LCC(N, T, K, [b]*K, p, q, r, g, alphas, betas, flag=0)
c_shares,_,_ = generate_shares_LCC(N, T, K, [c]*K, p, q, r, g, alphas, betas, flag=0)
ds,es = [],[]
for key in users_wb.keys():
    ds.append(( key , users_wb[key]['y'] - Field( share_ith(a_shares,key) )))
    es.append(( key , users_wb[key]['z'] - Field( share_ith(b_shares,key) )))
ds = reconstruct_secret_LCC(ds, q, betas, K)
es = reconstruct_secret_LCC(es, q, betas, K)
print('d,e:',d,e)
for i in range(K):
    d,e = ds[i],es[i]
    sigs = []
    for key in users_wb.keys():
        sigs.append(( key , d*e + d*share_ith(b_shares,key) + e*share_ith(a_shares,key) +share_ith(c_shares,key) ))
    yz_recon = reconstruct_secret_LCC(sigs, q, betas, K)
    yz_recon = yz_recon[i]
    shares = []
    for key in users_wb.keys():
        shares.append( (key , users_wb[key]['h_r']) )
    recon = reconstruct_secret_LCC(shares, q, betas, K)
    recon = recon[i]
    check = yz_recon - recon
    print("check",check)

2 2 3 6
0 + 6 t^1
2 6 1 2
0 + 11 t^1 + 4 t^2
users' shares: {'h_shares_0': 3, 'h_shares_1': 4, 'h_shares_2': 3, 'x_shares_0': 5, 'x_shares_1': 1, 'x_shares_2': 1}
**client 1**
f1: 1 10  g1: 1 10
**client 2**
f1: 6 0  g1: 12 11
**client 3**
f1: 5 2  g1: 0 8
**client 4**
f1: 12 6  g1: 2 5
users' shares: {'h_shares_0': 1, 'h_shares_1': 5, 'h_shares_2': 2, 'x_shares_0': 1, 'x_shares_1': 5, 'x_shares_2': 0, 'u1': 5, 'u2': 2, 'v1': 0, 'v2': 8 (mod 13), 'h_r': 8 (mod 13)}
users' shares: {'h_shares_0': 11, 'h_shares_1': 8, 'h_shares_2': 5, 'x_shares_0': 0, 'x_shares_1': 6, 'x_shares_2': 12, 'u1': 6, 'u2': 0, 'v1': 12, 'v2': 11 (mod 13), 'h_r': 11 (mod 13), 'y': 6, 'z': 12}
d,e: 12 9
check 0
check 0


In [13]:
# beaver's protocol
# consider additive sharing
# [y]i * [z]i = [yz]i

p,q,r,g = initial(13)
Field = FiniteField(p=q)
print('p,q,r,g:',p,q,r,g)
y,z = 2,3
a,b,c = Field(3),Field(5),Field(3*5)

y1,z1,a1,b1,c1 = [Field(random.randint(0, q-1)) for i in range(5)]
y2,z2,a2,b2,c2 = y-y1,z-z1,a-a1,b-b1,c-c1

d1, e1 = y1-a1, z1-b1
d2, e2 = y2-a2, z2-b2
d, e = d1+d2, e1+e2
sig1 = _divmod(d*e, 2, q) + d*b1+e*a1+c1 # share 1: [yz]1
sig2 = _divmod(d*e, 2, q) + d*b2+e*a2+c2 # share 2: [yz]2
print("sig1,sig2:",sig1,sig2)
print(sig1+sig2)

q = 13

q is prime

r = 4
p = 53

p is prime

Order of G is 13. This must be equal to q: 13

g = 44

p,q,r,g: 53 13 4 44
sig1,sig2: 5 1
6


In [14]:
def beaver(y1,z1,y2,z2,q):
    Field = FiniteField(p=q)
    a,b,c = Field(3),Field(5),Field(3*5)
    a1,b1,c1 = [Field(random.randint(0, q-1)) for i in range(3)]
    a2,b2,c2 = a-a1,b-b1,c-c1
    d1, e1 = y1-a1, z1-b1
    d2, e2 = y2-a2, z2-b2
    d, e = d1+d2, e1+e2
    yz1 = _divmod(d*e, 2, q) + d*b1+e*a1+c1 # share 1: [yz]1
    yz2 = _divmod(d*e, 2, q) + d*b2+e*a2+c2 # share 2: [yz]2
    return yz1,yz2

y,z = 2,3
y1,z1 = [Field(random.randint(0, q-1)) for i in range(2)]
y2,z2 = y-y1,z-z1
yz1,yz2 = beaver(y1,z1,y2,z2,q)
yz1,yz2,yz1+yz2

(6 (mod 13), 0 (mod 13), 6 (mod 13))

In [17]:
# back to additive sharing. SNIP itself
import random
Poly = polynomialsOver(Field)

x1, x2, x3 = 1,2,3
u1,u2 = Field(2),Field(2) # （1）
v1,v2 = Field(3),Field(6) # （2）

# compute poly over u1,u2 ; v1,v2
poly_f = Poly([2,0]) # depending on (1)
poly_g = Poly([0,3]) # depending on (2)
assert poly_f(1) == u1 and poly_f(2) == u2
assert poly_g(1) == v1 and poly_g(2) == v2
print(poly_f(1),poly_f(2),poly_g(1),poly_g(2))
h = poly_f*poly_g
print(h)
h_coefs = h.coefficients
h_shares = []
for c in h_coefs:
    rand = Field(random.randint(0, q-1))
    h_shares.append([(1,rand),(2,c-rand)])
x_shares = []
for x in [x1,x2,x3]:
    rand = Field(random.randint(0, q-1))
    x_shares.append([(1,rand),(2,x-rand)])
print("h_shares",h_shares,'\nx_shares',x_shares)

poly_h1 = Poly([h_shares[0][0][1], h_shares[1][0][1]])
poly_h2 = Poly([h_shares[0][1][1], h_shares[1][1][1]])
print("h shares:",poly_h1, ';',poly_h2)

# client idx = 1
print("**client 1**")
u11 = x_shares[1][0][1]
u21 = x_shares[0][0][1] *2
v11 = x_shares[2][0][1]
v21 = poly_h1(1)
print("f1:",u11,u21," g1:",v11,v21)

# client idx = 2
print("**client 2**")
u12 = x_shares[1][1][1]
u22 = x_shares[0][1][1] *2
v12 = x_shares[2][1][1]
v22 = poly_h2(1)
print("f2:",u12,u22," g2:",v12,v22)

# polynomial check
r = 5

# check_1 = ( _lagrange_interpolate(r, [1,2], [u11,u21], q) *
#            _lagrange_interpolate(r, [1,2], [v11,v21], q) - poly_h1(r)
#           )
# check_2 = ( _lagrange_interpolate(r, [1,2], [u12,u22], q) *
#            _lagrange_interpolate(r, [1,2], [v12,v22], q) - poly_h2(r)
#           )

# shares = [(1,check_1),(2,check_2)]
# recon = check_1 + check_2
# print("recon at r:",recon)

shares = [(1,poly_h1(2)), (2, poly_h2(2))]
recon = poly_h1(2) + poly_h2(2)
print("recon at out:",recon)

# Use Beaver
y1 = _lagrange_interpolate(r, [1,2], [u11,u21], q) 
y2 = _lagrange_interpolate(r, [1,2], [u12,u22], q) 
z1 = _lagrange_interpolate(r, [1,2], [v11,v21], q)
z2 = _lagrange_interpolate(r, [1,2], [v12,v22], q)
print(y1+y2,z1+z2)
yz1,yz2 = beaver(y1,z1,y2,z2,q)
check = yz1+yz2 - (poly_h1(r) + poly_h2(r))
print("check",check)

2 2 3 6
0 + 6 t^1
h_shares [[(1, 4 (mod 13)), (2, 9 (mod 13))], [(1, 3 (mod 13)), (2, 3 (mod 13))]] 
x_shares [[(1, 8 (mod 13)), (2, 6 (mod 13))], [(1, 0 (mod 13)), (2, 2 (mod 13))], [(1, 6 (mod 13)), (2, 10 (mod 13))]]
h shares: 4 + 3 t^1 ; 9 + 3 t^1
**client 1**
f1: 0 3  g1: 6 7
**client 2**
f2: 2 12  g2: 10 12
recon at out: 12
15 15
check 0
